# Transcriptional trajectories in mouse limb buds reveal the transition from anterior-posterior to proximal-distal patterning at early limb bud stage.
Desanlis I, Paul R, Kmita M. J Dev Biol. 2020;8(4):1–16. 
 
 PMID: [7768367](http://www.ncbi.nlm.nih.gov/pmc/articles/pmc7768367/)
 ***
 

## 0. Setting up workenvironment<a id="0"></a>

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
})

options(repr.plot.width = 16, repr.plot.height = 8)

## 5. E12.5<a id="28"></a>

### 5.1. Data analysis in Seurat<a id="29"></a>

In [ ]:
E12_5 <- readRDS(file = "/home/host_home/tom/Kmita/6-kmita/RDS_QC/E12_5.Rds")

In [ ]:
VlnPlot(E12_5, features = c("nCount_RNA", "nFeature_RNA", "subsets_Mito_percent"), 
        ncol = 3, group.by = "orig.ident", pt.size = 0)

#### 5.1.1. Feature selection

We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The procedure in Seurat v3 directly models the mean-variance relationship inherent in single-cell data, and is implemented in the `FindVariableFeatures` function. By default, it defaults to 2,000 features (genes) per dataset. These will be used in downstream analysis, like principal component analysis (PCA).

In [ ]:
E12_5 <- NormalizeData(E12_5, verbose = FALSE)
E12_5 <- FindVariableFeatures(E12_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(E12_5), 10)

In [ ]:
options(repr.plot.width=10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(E12_5)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

#### 5.1.2. Scaling the data

Next, we apply a linear transformation (scaling) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The ScaleData function:

- Shifts the expression of each gene, so that the mean expression across cells is 0
- Scales the expression of each gene, so that the variance across cells is 1
- This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate. This is superior to log-normalized counts, which overemphasize the influence of small count fluctuations.
- The results of this are stored in `object[["originalexp"]]@scale.data`

In [ ]:
all.genes <- rownames(E12_5)
E12_5 <- ScaleData(E12_5, features = all.genes)

#### 5.1.3 Linear dimensionality reduction: PCA

Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
E12_5 <- RunPCA(E12_5, features = VariableFeatures(object = E12_5), verbose = FALSE)

In [ ]:
DimPlot(E12_5, reduction = "pca", group.by = "Sample_Name") + xlab("PC 1") + ylab("PC 2")

#### 5.1.3. Determine dimensionality

To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a 'metafeature' that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many componenets should we choose to include? 

[Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8) introduced a resampling test inspired by the JackStraw procedure. They randomly permute a subset of the data (1% by default) and rerun PCA, constructing a 'null distribution' of feature scores, and repeat this procedure. 'Significant' PCs are defined as those who have a strong enrichment of low p-value features.

**NOTE: This process can take a long time for big datasets, and is therefore commented out for expediency. More approximate techniques such as a scree plot implemented in `ElbowPlot()` can be used to reduce computation time.**

The `JackStrawPlot` function provides a visualization tool for comparing the distribution of p-values for each PC with a uniform distribution (dashed line). 'Significant' PCs will show a strong enrichment of features with low p-values (solid curve above the dashed line).

In [ ]:
# object <- JackStraw(object, num.replicate = 100)
# object <- ScoreJackStraw(object, dims = 1:20)

In [ ]:
# JackStrawPlot(object, dims = 1:15)

An alternative heuristic method generates an 'Elbow plot': a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot function). 

In [ ]:
ElbowPlot(E12_5, 50)

### 5.2. Clustering<a id="30"></a>

Seurat v4 applies a graph-based clustering approach, building upon initial strategies in [Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. The new method embeds cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected 'quasi-cliques' or 'communities'.

As in [PhenoGraph](https://doi.org/10.1016/j.cell.2015.05.047), Seurat first constructs a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the `FindNeighbors` function, and takes as input the previously defined dimensionality of the dataset.

To cluster the cells, modularity optimization techniques such as the Louvain algorithm are then applied to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters` function implements this procedure and contains a resolution parameter that sets the 'granularity' of the downstream clustering, with increased values leading to a greater number of clusters. Optimal resolution often increases for larger datasets.

In [ ]:
E12_5 <- FindNeighbors(E12_5, dims = 1:40)
E12_5 <- FindClusters(E12_5, resolution = 0.15)

### 5.3. Non-linear dimensionality reduction: UMAP<a id="31"></a>

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, [it is recommended to use the same PCs](https://doi.org/10.1038/s41467-019-13056-x) as input to the clustering analysis.

In [ ]:
E12_5 <- RunUMAP(E12_5, dims = 1:40, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 8)

DimPlot(E12_5, reduction = "umap", label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E12_5_na.png" ,width=10, height=8)

### 5.4. Differential expression analysis<a id="32"></a>
#### 5.5.1. Global annotation by differential expression analysis<a id="33"></a>

Seurat identifies markers that define clusters via differential expression. By default, it identifes positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. `FindAllMarkers` automates this process for all clusters, but it is possible to test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the `thresh.test` argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significiant and the most highly differentially expressed features will likely still rise to the top.

In [ ]:
# Seurat default settings
E12.5.markers <- FindAllMarkers(E12_5, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
#dir.create("./6-kmita/analysis")
write.csv(E12.5.markers, file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/analysis/E12.5.markers.csv", quote = FALSE)

Cluster 0 + 1 + 2 + 3 + 4: mesenchymal cells 

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Mecom", "Prrx1", "Tbx5", "Prrx2", "Shox2", "Hoxd9"), min.cutoff = "q1")  

p1 + p2

#ggsave("./6-kmita/images/E12_5_c01234.png" ,width=18, height=8)

Cluster 5: ectoderm

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(E12_5, label = FALSE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Wnt3", "Fgf8", "En1", "Wnt7a"), min.cutoff = "q1") 

p1 + p2
#ggsave("./6-kmita/images/E12_5_c5.png" ,width=16, height=8)

Cluster 6: muscle cells

In [ ]:
options(repr.plot.width = 16)

p1 <- DimPlot(E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Pax3", "Lbx1", "Myf5", "Met"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E12_5_c6.png" ,width=16, height=8)

#find better markers

Cluster 7: endothelial cells  + bloodcells + ?

In [ ]:
p1 <- DimPlot(E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Pecam1", "Cd34", "Cd93", "Cdh5"), min.cutoff = "q1") 

p1 + p2

#ggsave("./6-kmita/images/E12_5_c7-1.png" ,width=16, height=8)

p3 <- DimPlot(E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p4 <- FeaturePlot(E12_5, c("Hbb-y", "Hba-x", "Hba-a1", "Hbb-bt"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p3 + p4

options(repr.plot.width = 16)
#ggsave("./6-kmita/images/E12_5_c7-2.png" ,width=16, height=8)

Cluster 8: Neural cells 

In [ ]:
p1 <- DimPlot(E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Nefl", "Mapt"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

options(repr.plot.width = 16)
#ggsave("./6-kmita/images/E12_5_c8.png" ,width=16, height=8)

Cluster 9: macrophages

In [ ]:
p1 <- DimPlot(E12_5) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E12_5, c("Cd3g", "Cd19", "Cd68", "Cxcr2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E12_5_c9.png" ,width=16, height=8)

In [ ]:
E12_5@meta.data$CellType <- Idents(E12_5)

In [ ]:
E12_5@meta.data$CellType <- as.character(E12_5@meta.data$CellType)

E12_5@meta.data$CellType[E12_5@meta.data$CellType == '0'] <- 'Mesenchyme'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '1'] <- 'Mesenchyme'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '2'] <- 'Mesenchyme'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '3'] <- 'Mesenchyme'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '4'] <- 'Mesenchyme'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '5'] <- 'Ectoderm'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '6'] <- 'Myocytes'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '7'] <- 'Doublets'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '8'] <- 'Neurons'
E12_5@meta.data$CellType[E12_5@meta.data$CellType == '9'] <- 'Immune cells'

E12_5@meta.data$CellType <- as.factor(E12_5@meta.data$CellType)

In [ ]:
new.cluster.ids <- c("mesenchyme","mesenchyme","mesenchyme","mesenchyme","mesenchyme", "ectoderm", "myocytes", "doublets", "neurons", "macrophages")
#new.cluster.ids <- c("0","1", "2", "3", "4")
names(new.cluster.ids) <- levels(E12_5)
E12_5 <- RenameIdents(E12_5, new.cluster.ids)
E12_5@meta.data$CellType = Idents(E12_5)

In [ ]:
new.cluster.ids <- c("Mesenchyme","Mesenchyme","Mesenchyme","Mesenchyme","Mesenchyme", "Ectoderm", "Myocytes", "Doublets", "Neurons", "Macrophages")
#new.cluster.ids <- c("0","1", "2", "3", "4")
names(new.cluster.ids) <- levels(E12_5)
E12_5 <- RenameIdents(E12_5, new.cluster.ids)
E12_5@meta.data$CellType = Idents(E12_5)

In [ ]:
options(repr.plot.width = 8)

DimPlot(E12_5, reduction = "umap", label = TRUE, group.by = "CellType") + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(E12_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

#### 5.5.2. Refinement of global annotations<a id="33.2"></a>

##### 5.5.2.1. Subsetting mesenchymal clusters<a id="34"></a>

HERE<a id="here"></a>

In [ ]:
E12_5 <- readRDS("./6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E12_5 <- subset(E12_5, idents = c("mesenchyme"))

In [ ]:
sc_mesenchymal_E12_5

In [ ]:
sc_mesenchymal_E12_5 <- ScaleData(sc_mesenchymal_E12_5)

In [ ]:
sc_mesenchymal_E12_5 <- RunPCA(sc_mesenchymal_E12_5, features = VariableFeatures(object = sc_mesenchymal_E12_5), verbose = FALSE)

In [ ]:
options(repr.plot.width=10)
DimPlot(sc_mesenchymal_E12_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_mesenchymal_E12_5, 50)

In [ ]:
sc_mesenchymal_E12_5 <- FindNeighbors(sc_mesenchymal_E12_5, dims = 1:25, verbose = FALSE)
sc_mesenchymal_E12_5 <- FindClusters(sc_mesenchymal_E12_5, resolution = 3, verbose = FALSE)

In [ ]:
sc_mesenchymal_E12_5 <- RunUMAP(sc_mesenchymal_E12_5, dims = 1:25, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)
DimPlot(sc_mesenchymal_E12_5, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

ggsave("./6-kmita/images/E12_5_m_na.png" ,width=9, height=8)

In [ ]:
markers_12_5_mesenchyme <- FindAllMarkers(sc_mesenchymal_E12_5)

In [ ]:
saveRDS(markers_12_5_mesenchyme, file = "./6-kmita/analysis/markers_12_5_mesenchyme.Rds")

In [ ]:
.markers_12_5_mesenchyme <- readRDS(file = "./6-kmita/analysis/markers_12_5_mesenchyme.Rds")

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df_m %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

markers_12_5_mesenchyme[markers_12_5_mesenchyme$cluster == 40,] %>%
slice_max(n = 50, order_by = avg_log2FC)

Distal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("22","3"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1

ggsave("./6-kmita/images/E12_5_m_distal-1.png" ,width=9.5, height=8)
options(repr.plot.width = 17, repr.plot.height = 8)
p2

ggsave("./6-kmita/images/E12_5_m_distal-2.png" ,width=17, height=8)

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("22","3"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("T")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1


options(repr.plot.width = 17, repr.plot.height = 8)
p2



In [ ]:
options(repr.plot.width = 28, repr.plot.height = 8)
FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5", "Sox9"), blend = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

ggsave("./6-kmita/images/E12_5_Gdf5_Sox9.png" ,width=28, height=8)

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("22","3"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1
options(repr.plot.width = 17, repr.plot.height = 8)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("0","1","2","5","7","8","10","11","12","13","14","15","16","18","19","20","21","26","27","28","29","30","36","37","39","40","41","6"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1
options(repr.plot.width = 20, repr.plot.height = 8)
p2

Look at intermediate 

In [ ]:
#                                     0         1           2         3          4          5        6         7         8         9         10        11       12         13        14         15      16         17        18        19        20       21         22       23         24        25       26        27        28          29       30         31         32        33        34          35       36         37        38       39         40     41   
palette.sc_mesenchymal_E12_5 <- c("#e31a1c","#e31a1c", "#e31a1c", "#808080", "#808080","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#808080","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#808080","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#808080","#808080","#808080","#808080","#e31a1c","#e31a1c","#e31a1c", "#e31a1c","#e31a1c","#808080", "#808080","#808080","#808080","#808080","#e31a1c","#e31a1c", "#808080","#e31a1c","#e31a1c","#e31a1c")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + xlab("UMAP 1") + ylab("UMAP 2")
options(repr.plot.width = 20, repr.plot.height = 8)
FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

>Explanation colors:
>
> <span style="color:#e31a1c">#e31a1c (red) -> distal mesenchyme</span>
>
> <span style="color:#808080">#808080 (gray) -> no annotation</span>

Proximal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("41"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9", "Shox2", "Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 16)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("3","4","6","7","9","17","22","23","34","35","38"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9", "Shox2", "Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 
p2
options(repr.plot.width = 20, repr.plot.height = 8)
p3

#INTERMEDIATE

In [ ]:
#                                     0         1           2         3          4          5        6         7         8         9         10        11       12         13        14         15      16         17        18        19        20       21         22       23         24        25       26        27        28          29       30         31         32        33        34       35         36         37        38       39         40       41   
palette.sc_mesenchymal_E12_5 <- c("#e31a1c","#e31a1c", "#e31a1c", "#08519c", "#08519c","#e31a1c","#08519c","#08519c","#e31a1c","#08519c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#08519c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#08519c","#08519c","#08519c","#08519c","#e31a1c","#e31a1c","#e31a1c", "#08519c","#e31a1c","#08519c", "#08519c","#08519c","#08519c","#08519c","#e31a1c","#e31a1c", "#08519c","#e31a1c","#e31a1c","#e31a1c")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + xlab("UMAP 1") + ylab("UMAP 2")

>Explanation colors:
>
> <span style="color:#e31a1c">#e31a1c (red) -> distal mesenchyme</span>
>
> <span style="color:#08519c">#08519c (blue) -> proximal mesenchyme </span>
>
> <span style="color:#808080">#808080 (gray) -> no annotation</span>

Cartilage

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("41"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1","Sox5","Sox6")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Prrx1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 16, repr.plot.height = 6)
p1 + p3

ggsave("./6-kmita/images/E12_5_m_c-1.png" ,width=16, height=6)
options(repr.plot.width = 16, repr.plot.height = 12)
p2
ggsave("./6-kmita/images/E12_5_m_c-1.png" ,width=16, height=12)


In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("10","29","30","40", "21", "18"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox5","Prrx1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 16, repr.plot.height = 6)
p2
ggsave("./6-kmita/images/E12_5_m_c-1.png" ,width=16, height=6)
options(repr.plot.width = 8, repr.plot.height = 6)
p1
ggsave("./6-kmita/images/E12_5_m_c-2.png" ,width=8, height=6)
options(repr.plot.width = 16, repr.plot.height = 6)
p3
ggsave("./6-kmita/images/E12_5_m_c-3.png" ,width=16, height=6)

In [ ]:
#                                     0         1           2         3          4          5        6         7         8         9         10        11       12         13        14          15      16         17        18        19        20       21         22       23         24        25       26        27        28          29       30         31         32        33        34          35       36         37        38       39         40    41    
palette.sc_mesenchymal_E12_5 <- c("#e31a1c","#e31a1c", "#00ff00", "#cc00cc", "#cc00cc","#e31a1c","#08519c","#08519c","#e31a1c","#cc00cc","#00ffff","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#00ff00","#e31a1c","#08519c","#00ffff","#e31a1c","#e31a1c","#00ffff","#cc00cc","#08519c","#08519c","#08519c","#e31a1c","#e31a1c","#00ff00", "#ff8c00","#00ffff","#cc00cc", "#08519c","#08519c","#08519c","#08519c","#e31a1c","#e31a1c", "#cc00cc","#00ff00","#00ffff","#00ff00")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + xlab("UMAP 1") + ylab("UMAP 2")

> <span style="color:#808080">#808080 (gray) -> no annotation</span>

>Explanation colors:
>
> <span style="color:#e31a1c">#e31a1c (red) -> distal cells</span>
>
> <span style="color:#08519c">#08519c (blue) -> proximal cells</span>
>
> <span style="color:#ffea00">#ffea00 (yellow) ->  </span>
>
><span style="color:#00ff00">#00ff00 (green) -> distal cartilage precursors </span>
>
><span style="color:#cc00cc">#cc00cc (purple) -> proximal cartilage precursors</span>
>
><span style="color:#00ffff">#00ffff (cyan) -> distal cartilage  </span>
>
><span style="color:#ff8c00">#ff8c00 (orange) -> proximal cartilage  </span>

Digits/ inter-digit precursors

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("21"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Aldh1a2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 16, repr.plot.height = 6)
p1 + p2

ggsave("./6-kmita/images/E12_5_di.png" ,width=16, height=6)

In [ ]:
p1 <- DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Scx", "Dcn", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("23"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Scx", "Dcn", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#Check cluster 23

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("1","17","19","24","32","34", "7"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Scx", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

ggsave("./6-kmita/images/E12_5_m_t.png" ,width=20, height=8)

In [ ]:
#                                     0         1           2         3          4          5        6         7         8         9         10        11       12         13        14          15      16         17        18        19        20       21         22       23         24        25       26        27        28          29       30         31         32        33        34          35       36         37        38       39         40    41    
palette.sc_mesenchymal_E12_5 <- c("#e31a1c","#f6acff", "#00ff00", "#cc00cc", "#cc00cc","#e31a1c","#08519c","#616d00","#e31a1c","#cc00cc","#00ffff","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#00ff00","#e31a1c","#616d00","#00ffff","#f6acff","#e31a1c","#ffea00","#cc00cc","#08519c","#616d00","#08519c","#e31a1c","#e31a1c","#00ff00", "#ff8c00","#00ffff","#cc00cc", "#616d00","#08519c","#616d00","#08519c","#e31a1c","#e31a1c", "#cc00cc","#00ff00","#00ffff","#00ff00")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + xlab("UMAP 1") + ylab("UMAP 2")

> <span style="color:#808080">#808080 (gray) -> no annotation</span>

>Explanation colors:
>
> <span style="color:#e31a1c">#e31a1c (red) -> distal cells</span>
>
> <span style="color:#08519c">#08519c (blue) -> proximal cells</span>
>
> <span style="color:#ffea00">#ffea00 (yellow) -> digit and inter-digit precursors  </span>
>
><span style="color:#00ff00">#00ff00 (green) -> distal cartilage precursors </span>
>
><span style="color:#cc00cc">#cc00cc (purple) -> proximal cartilage precursors</span>
>
><span style="color:#00ffff">#00ffff (cyan) -> distal cartilage  </span>
>
><span style="color:#ff8c00">#ff8c00 (orange) -> proximal cartilage  </span>
>
><span style="color:#f6acff">#f6acff (pink) -> distal tenocyte precursors  </span>
>
><span style="color:#616d00">#616d00 (dark green) -> proximal cartilage  </span>


Intermediate mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("41"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
options(repr.plot.width = 10, repr.plot.height = 8)
p3

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("3","4","6","9","17","22","31","32","35","36","38"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9", "Shox2", "Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
options(repr.plot.width = 20, repr.plot.height = 8)
p3

#INTERMEDIATE

Joint

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("41"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5")) + xlab("UMAP 1") + ylab("UMAP 2")


options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
options(repr.plot.width = 10, repr.plot.height = 8)
p3

38 29 40 21 22

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("distal cartilage"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5")) + xlab("UMAP 1") + ylab("UMAP 2")


options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("38", "29", "40", "21", "22"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
ggsave("./6-kmita/images/E12_5_joint.png" ,width=16, height=6)

Look at Gdf5, Ihh (pre-hypertrophe), Pthr1, 

Distal, intermediate and proximal cells

In [ ]:
#                                     0         1           2         3          4          5        6         7         8         9         10        11       12         13        14          15      16         17        18        19        20       21         22       23         24        25       26        27        28          29       30         31         32        33        34          35       36         37        38       39         40    41    
palette.sc_mesenchymal_E12_5 <- c("#e31a1c","#e31a1c", "#e31a1c", "#ffea00", "#ffea00","#e31a1c","#ffea00","#08519c","#e31a1c","#ffea00","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#ffea00","#e31a1c","#e31a1c","#e31a1c","#e31a1c","#ffea00","#08519c","#08519c","#08519c","#e31a1c","#e31a1c","#e31a1c", "#08519c","#e31a1c","#ffea00", "#ffea00","#08519c","#08519c","#ffea00","#ffea00","#e31a1c", "#ffea00","#e31a1c","#e31a1c","#e31a1c")

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = TRUE, cols = palette.sc_mesenchymal_E12_5) + xlab("UMAP 1") + ylab("UMAP 2")

ggsave("./6-kmita/images/E12_5_m_dip-1.png" ,width=10, height=8)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9", "Shox2", "Meis1", "Meis2")) + xlab("UMAP 1") + ylab("UMAP 2")
ggsave("./6-kmita/images/E12_5_m_dip-2.png" ,width=10, height=8)

FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
ggsave("./6-kmita/images/E12_5_m_dip-3.png" ,width=10, height=8)

p1 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13")) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- p1 - p2
options(repr.plot.width = 20, repr.plot.height = 8)
p3
ggsave("./6-kmita/images/E12_5_m_dip-4.png" ,width=20, height=8)

>Explanation colors:
>
> <span style="color:#e31a1c">#e31a1c (red) -> distal cells</span>
>
> <span style="color:#08519c">#08519c (blue) -> proximal cells</span>
>
> <span style="color:#ffea00">#ffea00 (yellow) -> intermediate cells </span>

In [ ]:
#                     0     1    2     3     4     5   6    7    8    9   10   11  12  13  14   15   16   17   18     19   20   21   22   23  24   25  26  27   28   29   30   31    32   33     34   35 36   37    38   39    40   41 
new.cluster.ids <- c("D","DPT","DPC","IPC", "IPC","D","I","PPT","D","IPC","DC","D","D","D","D","DPC","D","IPT","DC","DPT","D","DP","IPC","P","PPT","P","D","D","DPC","PC","DC","IPC","IPT","P","PPT","I","I","PPC","IC","DPC","DC","DPC")
names(new.cluster.ids) <- levels(sc_mesenchymal_E12_5)
sc_mesenchymal_E12_5 <- RenameIdents(sc_mesenchymal_E12_5, new.cluster.ids)
sc_mesenchymal_E12_5@meta.data$labels_mesenchyme_E12_5 = Idents(sc_mesenchymal_E12_5)

In [ ]:
new.cluster.ids <- c("distal mesenchyme","distal tenocyte precursors", "distal cartilage precursors","intermediate cartilage precursors","intermediate mesenchyme","proximal tenocyte precursors","distal cartilage", "intermediate tenocyte precursors", "distal joint precursors", "proximal mesenchyme", "proximal cartilage","proximal cartilage precursors", "intermediate cartilage")
names(new.cluster.ids) <- levels(sc_mesenchymal_E12_5)
sc_mesenchymal_E12_5 <- RenameIdents(sc_mesenchymal_E12_5, new.cluster.ids)
sc_mesenchymal_E12_5@meta.data$labels_mesenchyme_E12_5 = Idents(sc_mesenchymal_E12_5)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E12_5, label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")

ggsave("./6-kmita/images/E12_5_m_a.png" ,width=16, height=8)

In [ ]:
saveRDS(sc_mesenchymal_E12_5, file = "./6-kmita/RDataSessions/sc_mesenchymal_E12_5_annotated_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E12_5 <- readRDS("./6-kmita/RDataSessions/sc_mesenchymal_E12_5_annotated_SeuratObject.Rds")

In [ ]:
#Dotplot
features <- c("Prrx1","Prrx2","Tbx5","Meis1","Hoxa9","Hoxa11","Hoxd11","Hoxa13","Hoxd13","Shh","Sox9","Scx","Gdf5","Fgf8","Wnt7a","Pax3","Pecam1","Sox10","Hba-a1")

options(repr.plot.width = 12, repr.plot.height = 5)

DotPlot(sc_mesenchymal_E12_5, features = features,col.min = 0) + RotatedAxis()

#### 5.5.2. Refinement of global annotations<a id="33.2"></a>
##### 5.5.2.1. Subsetting mesenchymal clusters - REDO

HERE<a id="here"></a>

In [ ]:
E12_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E12_5) <- E12_5@meta.data$CellType

In [ ]:
sc_mesenchymal_E12_5 <- subset(E12_5, idents = c("Mesenchyme"))

In [ ]:
sc_mesenchymal_E12_5

In [ ]:
sc_mesenchymal_E12_5 <- NormalizeData(sc_mesenchymal_E12_5, verbose = FALSE)
sc_mesenchymal_E12_5 <- FindVariableFeatures(sc_mesenchymal_E12_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_mesenchymal_E12_5), 10)

In [ ]:
sc_mesenchymal_E12_5 <- ScaleData(sc_mesenchymal_E12_5)

In [ ]:
sc_mesenchymal_E12_5 <- RunPCA(sc_mesenchymal_E12_5, features = VariableFeatures(object = sc_mesenchymal_E12_5), verbose = FALSE)

In [ ]:
options(repr.plot.width=10)
DimPlot(sc_mesenchymal_E12_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_mesenchymal_E12_5, 50)

In [ ]:
sc_mesenchymal_E12_5 <- FindNeighbors(sc_mesenchymal_E12_5, dims = 1:25, verbose = FALSE)
sc_mesenchymal_E12_5 <- FindClusters(sc_mesenchymal_E12_5, resolution = 3, verbose = FALSE)

In [ ]:
sc_mesenchymal_E12_5 <- RunUMAP(sc_mesenchymal_E12_5, dims = 1:25, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)
DimPlot(sc_mesenchymal_E12_5, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
#markers_12_5_mesenchyme <- FindAllMarkers(sc_mesenchymal_E12_5)

In [ ]:
#saveRDS(markers_12_5_mesenchyme, file = "./6-kmita/analysis/markers_12_5_mesenchyme.Rds")

In [ ]:
#.markers_12_5_mesenchyme <- readRDS(file = "./6-kmita/analysis/markers_12_5_mesenchyme.Rds")

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df_m %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

#markers_12_5_mesenchyme[markers_12_5_mesenchyme$cluster == 40,] %>%
#slice_max(n = 50, order_by = avg_log2FC)

Proximal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("21","32"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Meis1","Meis2","Hoxd9",'Hoxa9')) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("21","16","27","32","9","31"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Meis1","Hoxa9","Hoxd9")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

Intermediate mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("18"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa11", "Hoxd11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 7, repr.plot.height = 5.5)
p1 
options(repr.plot.width = 12, repr.plot.height = 5.5)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("20","24","34","5","22","4",'9','17','28','15','10','4','2',"27","31","16",'30','33'))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa11", "Hoxd11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 7, repr.plot.height = 5.5)
p1 
options(repr.plot.width = 12, repr.plot.height = 5.5)
p2

Distal mesenchyme

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("19"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 7, repr.plot.height = 5.5)
p1 
options(repr.plot.width = 12, repr.plot.height = 5.5)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c())
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9.5, repr.plot.height = 8)
p1
options(repr.plot.width = 17, repr.plot.height = 8)
p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("36"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Hoxd9","Hoxa11","Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 28, repr.plot.height = 11)
p1 + p2

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType_PID <- Idents(sc_mesenchymal_E12_5)

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType_PID <- as.character(sc_mesenchymal_E12_5@meta.data$CellType_PID)

sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '21'] <- 'Proximal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '32'] <- 'Proximal cells'

sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '20'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '24'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '34'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '5'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '22'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '4'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '9'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '17'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '28'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '15'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '10'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '4'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '2'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '27'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '31'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '16'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '30'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '33'] <- 'Intermediate cells'

sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '0'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '1'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '11'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '12'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '13'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '14'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '18'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '19'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '23'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '25'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '26'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '29'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '3'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '35'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '36'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '37'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '6'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '7'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '8'] <- 'Distal cells'

sc_mesenchymal_E12_5@meta.data$CellType_PID <- as.factor(sc_mesenchymal_E12_5@meta.data$CellType_PID)

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 11)
DimPlot(sc_mesenchymal_E12_5, group.by = "CellType_PID") + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 11)
DimPlot(sc_mesenchymal_E12_5, label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

Cartilage precursors

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("0"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1","Sox5","Sox6","Prrx2")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Prrx1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 16, repr.plot.height = 6)
p1 + p3
options(repr.plot.width = 16, repr.plot.height = 18)
p2


In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("8","28","15","10","35"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1","Sox5","Sox6","Prrx2")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Prrx1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 16, repr.plot.height = 6)
p1 + p3
options(repr.plot.width = 16, repr.plot.height = 18)
p2


Cartilage

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("0"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1","Sox5","Sox6",'Col10a1','Pthlh')) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("0",'13','11','26'))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Col2a1","Sox5","Sox6",'Col10a1','Pthlh')) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
#Resting zone chondrocytes
g <- WhichCells(sc_mesenchymal_E12_5, idents = c('13',"0"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Fgfr3", "Pcp4", "Pthlh", "Sfrp5", "Spon1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
#Proliferative chondrocytes
g <- WhichCells(sc_mesenchymal_E12_5, idents = c('26','11'))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Sox9", "Mki67", "Top2a", "Pth1r")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

Tenocyte precursors

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("21","9","16","34","5","20","24","17","32","22"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g),label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Tnmd", "Scx", "Prrx1", "Prrx2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#Check cluster 23

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("1","17","19","24","32","34", "7"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Scx", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E12_5_m_t.png" ,width=20, height=8)

Joint

In [ ]:
g <- WhichCells(sc_mesenchymal_E12_5, idents = c("25"))
p1 <- DimPlot(sc_mesenchymal_E12_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E12_5, c("Smoc2", "Dact2", "Wnt4", "Wnt16")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5","Sox9"), blend = TRUE)
p4 <- FeaturePlot(sc_mesenchymal_E12_5, c("Gdf5","Prrx1"), blend = TRUE)

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
options(repr.plot.width = 20, repr.plot.height = 8)
p3
p4

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType <- Idents(sc_mesenchymal_E12_5)

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == ''] <- 'Limb bud mesenchyme'

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType <- as.character(sc_mesenchymal_E12_5@meta.data$CellType)

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '8'] <- 'Chondroprogenitors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '28'] <- 'Chondroprogenitors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '15'] <- 'Chondroprogenitors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '10'] <- 'Chondroprogenitors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '35'] <- 'Chondroprogenitors'

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '13'] <- 'Resting zone chondrocytes'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '0'] <- 'Resting zone chondrocytes'

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '26'] <- 'Proliferative chondrocytes'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '11'] <- 'Proliferative chondrocytes'

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '25'] <- 'Joint precursors'

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '21'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '9'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '16'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '34'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '5'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '20'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '24'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '17'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '32'] <- 'Tenocyte precursors'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '22'] <- 'Tenocyte precursors'

sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '1'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '12'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '14'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '18'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '19'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '2'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '23'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '27'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '29'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '3'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '30'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '31'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '33'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '36'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '37'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '4'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '6'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E12_5@meta.data$CellType[sc_mesenchymal_E12_5@meta.data$CellType == '7'] <- 'Distal limb bud mesenchyme'

sc_mesenchymal_E12_5@meta.data$CellType <- as.factor(sc_mesenchymal_E12_5@meta.data$CellType)

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '21'] <- 'Proximal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '32'] <- 'Proximal cells'

sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '20'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '24'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '34'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '5'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '22'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '4'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '9'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '17'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '28'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '15'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '10'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '4'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '2'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '27'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '31'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '16'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '30'] <- 'Intermediate cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '33'] <- 'Intermediate cells'

sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '0'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '1'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '11'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '12'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '13'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '14'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '18'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '19'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '23'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '25'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '26'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '29'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '3'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '35'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '36'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '37'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '6'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '7'] <- 'Distal cells'
sc_mesenchymal_E12_5@meta.data$CellType_PID[sc_mesenchymal_E12_5@meta.data$CellType_PID == '8'] <- 'Distal cells'

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 11)
DimPlot(sc_mesenchymal_E12_5, group.by = "CellType") + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
saveRDS(sc_mesenchymal_E12_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E12_5_annotated_SeuratObject.Rds")

##### 5.4.2.2. Subsetting ectodermal cluster<a id="35"></a>

In [ ]:
E12_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E12_5) <- E12_5@meta.data$CellType

In [ ]:
sc_ectoderm_E12_5 <- subset(E12_5, idents = c("Ectoderm"))

In [ ]:
sc_ectoderm_E12_5

In [ ]:
sc_ectoderm_E12_5 <- NormalizeData(sc_ectoderm_E12_5, verbose = FALSE)
sc_ectoderm_E12_5 <- FindVariableFeatures(sc_ectoderm_E12_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_ectoderm_E12_5), 10)

In [ ]:
sc_ectoderm_E12_5 <- ScaleData(sc_ectoderm_E12_5)

In [ ]:
sc_ectoderm_E12_5 <- RunPCA(sc_ectoderm_E12_5, features = VariableFeatures(object = sc_ectoderm_E12_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_ectoderm_E12_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_ectoderm_E12_5, 50)

In [ ]:
sc_ectoderm_E12_5 <- FindNeighbors(sc_ectoderm_E12_5, dims = 1:40)
sc_ectoderm_E12_5 <- FindClusters(sc_ectoderm_E12_5, resolution = 0.4)

In [ ]:
sc_ectoderm_E12_5 <- RunUMAP(sc_ectoderm_E12_5, dims = 1:40, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E12_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_ectoderm_E12_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 3,] %>%
slice_max(n = 20, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

Skin

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_ectoderm_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E12_5, c("Krt5", "Krt10","Krt7",'Anxa3','Sox9','Prrx1',"Col2a1",'Hoxa9','Fgf8')) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_e_neurons.png" ,width=20, height=8)

In [ ]:
FeaturePlot(sc_ectoderm_E12_5, c('Sox9',"Krt5"), blend = TRUE)

Ectoderm

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_ectoderm_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E12_5, c("Wnt7a", "En1")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_e_neurons.png" ,width=20, height=8)

AER

In [ ]:
options(repr.plot.width = 20)

p1 <- DimPlot(sc_ectoderm_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E12_5, c("Fgf8", "Bmp4", "Dlx5", "Rspo2")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

In [ ]:
sc_ectoderm_E12_5@meta.data$CellType <- Idents(sc_ectoderm_E12_5)

In [ ]:
sc_ectoderm_E12_5@meta.data$CellType <- as.character(sc_ectoderm_E12_5@meta.data$CellType)

sc_ectoderm_E12_5@meta.data$CellType[sc_ectoderm_E12_5@meta.data$CellType == '0'] <- 'Ectoderm'
sc_ectoderm_E12_5@meta.data$CellType[sc_ectoderm_E12_5@meta.data$CellType == '1'] <- 'Ectoderm'
sc_ectoderm_E12_5@meta.data$CellType[sc_ectoderm_E12_5@meta.data$CellType == '2'] <- 'AER'
sc_ectoderm_E12_5@meta.data$CellType[sc_ectoderm_E12_5@meta.data$CellType == '3'] <- 'Doublets'

sc_ectoderm_E12_5@meta.data$CellType <- as.factor(sc_ectoderm_E12_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E12_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E12_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_ectoderm_E12_5, file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E12_5_annotated_SeuratObject.Rds")

##### 4.4.2.3. Subsetting blood and endothilial cluster<a id="35-2"></a>

In [ ]:
E12_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E12_5) <- E12_5@meta.data$CellType

In [ ]:
sc_BE_E12_5 <- subset(E12_5, idents = c("Doublets"))

In [ ]:
sc_BE_E12_5

In [ ]:
sc_BE_E12_5 <- NormalizeData(sc_BE_E12_5, verbose = FALSE)
sc_BE_E12_5 <- FindVariableFeatures(sc_BE_E12_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_BE_E12_5), 10)

In [ ]:
sc_BE_E12_5 <- ScaleData(sc_BE_E12_5)

In [ ]:
sc_BE_E12_5 <- RunPCA(sc_BE_E12_5, features = VariableFeatures(object = sc_BE_E12_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_BE_E12_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_BE_E12_5, 50)

In [ ]:
sc_BE_E12_5 <- FindNeighbors(sc_BE_E12_5, dims = 1:40)
sc_BE_E12_5 <- FindClusters(sc_BE_E12_5, resolution = 1)

In [ ]:
sc_BE_E12_5 <- RunUMAP(sc_BE_E12_5, dims = 1:40, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_BE_E12_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_BE_E12_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 1,] %>%
slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

Endothilium

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_BE_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_BE_E12_5, c("Pecam1", "Cd34", "Cd93", "Cdh5", "Emcn"), min.cutoff = "q1") 

p1 + p2

Erythrocytes

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_BE_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_BE_E12_5, c("Hbb-y", "Hba-x", "Hba-a1", "Hbb-bt"), min.cutoff = "q1")


p1 + p2

In [ ]:
sc_BE_E12_5@meta.data$CellType <- Idents(sc_BE_E12_5)

In [ ]:
sc_BE_E12_5@meta.data$CellType <- as.character(sc_BE_E12_5@meta.data$CellType)

sc_BE_E12_5@meta.data$CellType[sc_BE_E12_5@meta.data$CellType == '0'] <- 'Vascular endothelial cells'
sc_BE_E12_5@meta.data$CellType[sc_BE_E12_5@meta.data$CellType == '1'] <- 'Erythroid cells'

sc_BE_E12_5@meta.data$CellType <- as.factor(sc_BE_E12_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_BE_E12_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_BE_E12_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_BE_E12_5_annotated_SeuratObject.Rds")

##### 4.4.2.4. Subsetting myocyte cluster<a id="35-2"></a>

In [ ]:
E12_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E12_5) <- E12_5@meta.data$CellType

In [ ]:
sc_myocytes_E12_5 <- subset(E12_5, idents = c("Myocytes"))

In [ ]:
sc_myocytes_E12_5

In [ ]:
sc_myocytes_E12_5 <- NormalizeData(sc_myocytes_E12_5, verbose = FALSE)
sc_myocytes_E12_5 <- FindVariableFeatures(sc_myocytes_E12_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_myocytes_E12_5), 10)

In [ ]:
sc_myocytes_E12_5 <- ScaleData(sc_myocytes_E12_5)

In [ ]:
sc_myocytes_E12_5 <- RunPCA(sc_myocytes_E12_5, features = VariableFeatures(object = sc_myocytes_E12_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_myocytes_E12_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_myocytes_E12_5, 50)

In [ ]:
sc_myocytes_E12_5 <- FindNeighbors(sc_myocytes_E12_5, dims = 1:25)
sc_myocytes_E12_5 <- FindClusters(sc_myocytes_E12_5, resolution = 1)

In [ ]:
sc_myocytes_E12_5 <- RunUMAP(sc_myocytes_E12_5, dims = 1:40, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_myocytes_E12_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_myocytes_E12_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 1,] %>%
slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

Myogenic stem cells

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_myocytes_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_myocytes_E12_5, c("Pax7","Myf5"), min.cutoff = "q1") 

p1 + p2

Myogenic stem cells

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_myocytes_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_myocytes_E12_5, c("Pax7", "Notch", "Itm2a", "Myf5", "Pax3", "Myod"), min.cutoff = "q1") 

p1 + p2

Skeletal muscle cell

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_myocytes_E12_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_myocytes_E12_5, c("Myhc", "Mrf4", 'Tnnt1','Tnnt3'), min.cutoff = "q1") 

p1 + p2

In [ ]:
sc_myocytes_E12_5@meta.data$CellType <- Idents(sc_myocytes_E12_5)

In [ ]:
sc_myocytes_E12_5@meta.data$CellType <- as.character(sc_myocytes_E12_5@meta.data$CellType)

sc_myocytes_E12_5@meta.data$CellType[sc_myocytes_E12_5@meta.data$CellType == '0'] <- 'Muscle progenitors'
sc_myocytes_E12_5@meta.data$CellType[sc_myocytes_E12_5@meta.data$CellType == '1'] <- 'Skeletal muscle cells'
sc_myocytes_E12_5@meta.data$CellType[sc_myocytes_E12_5@meta.data$CellType == '2'] <- 'Muscle progenitors'
sc_myocytes_E12_5@meta.data$CellType[sc_myocytes_E12_5@meta.data$CellType == '3'] <- 'Muscle progenitors'

sc_myocytes_E12_5@meta.data$CellType <- as.factor(sc_myocytes_E12_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_myocytes_E12_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_myocytes_E12_5, file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_myocytes_E12_5_annotated_SeuratObject.Rds")

#### 5.4.2. Updating annotations<a id="36"></a>

In [ ]:
E12_5 <- readRDS("/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E12_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E12_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E12_5_annotated_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E12_5@meta.data$CellType_m <- sc_mesenchymal_E12_5@meta.data$CellType
sc_mesenchymal_E12_5@meta.data$CellType <- NULL

In [ ]:
sc_ectoderm_E12_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E12_5_annotated_SeuratObject.Rds")

In [ ]:
sc_ectoderm_E12_5@meta.data$CellType_e <- sc_ectoderm_E12_5@meta.data$CellType
sc_ectoderm_E12_5@meta.data$CellType <- NULL

In [ ]:
sc_BE_E12_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_BE_E12_5_annotated_SeuratObject.Rds")

In [ ]:
sc_BE_E12_5@meta.data$CellType_be <- sc_BE_E12_5@meta.data$CellType
sc_BE_E12_5@meta.data$CellType <- NULL

In [ ]:
sc_myocytes_E12_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_myocytes_E12_5_annotated_SeuratObject.Rds")

In [ ]:
sc_myocytes_E12_5@meta.data$CellType_my <- sc_myocytes_E12_5@meta.data$CellType
sc_myocytes_E12_5@meta.data$CellType <- NULL

In [ ]:
meta1 <- E12_5@meta.data
meta2 <- sc_mesenchymal_E12_5@meta.data

In [ ]:
df <- split(meta1, meta1$CellType)

In [ ]:
mesenchyme <- rownames(df$Mesenchyme)

In [ ]:
mesenchyme_meta <- meta2[mesenchyme, c("CellType_m", "orig.ident")]
mesenchyme_meta$orig.ident <- NULL

head(mesenchyme_meta)

In [ ]:
df$Mesenchyme <- cbind(df$Mesenchyme, mesenchyme_meta)
df$Mesenchyme$CellType <- df$Mesenchyme$CellType_m
df$Mesenchyme$CellType_m <- NULL

In [ ]:
str(df)

In [ ]:
ectoderm <- rownames(df$Ectoderm)

In [ ]:
meta2 <- sc_ectoderm_E12_5@meta.data
ectoderm_meta <- meta2[ectoderm, c("CellType_e", "orig.ident")]
ectoderm_meta$orig.ident <- NULL

head(ectoderm_meta)

In [ ]:
df$Ectoderm <- cbind(df$Ectoderm, ectoderm_meta)
df$Ectoderm$CellType <- df$Ectoderm$CellType_e
df$Ectoderm$CellType_e <- NULL

In [ ]:
BE <- rownames(df$Doublets)

In [ ]:
meta2 <- sc_BE_E12_5@meta.data
BE_meta <- meta2[BE, c("CellType_be", "orig.ident")]
BE_meta$orig.ident <- NULL

head(BE_meta)

In [ ]:
df$Doublets <- cbind(df$Doublets, BE_meta)
df$Doublets$CellType <- df$Doublets$CellType_be
df$Doublets$CellType_be <- NULL

In [ ]:
myocytes <- rownames(df$Myocytes)

In [ ]:
meta2 <- sc_myocytes_E12_5@meta.data
myocytes_meta <- meta2[myocytes, c("CellType_my", "orig.ident")]
myocytes_meta$orig.ident <- NULL

head(myocytes_meta)

In [ ]:
df$Myocytes <- cbind(df$Myocytes, myocytes_meta)
df$Myocytes$CellType <- df$Myocytes$CellType_my
df$Myocytes$CellType_my <- NULL

In [ ]:
df$'Immune cells'$Barcode <- rownames(df$'Immune cells')
df$Neurons$Barcode <- rownames(df$Neurons)

In [ ]:
df$Mesenchyme$Barcode = rownames(df$Mesenchyme)
df$Ectoderm$Barcode = rownames(df$Ectoderm)
df$Doublets$Barcode = rownames(df$Doublets)
df$Myocytes$Barcode = rownames(df$Myocytes)
df$'Immune cells'$Barcode = rownames(df$'Immune cells')
df$Neurons$Barcode = rownames(df$Neurons)

**Make sure all dataframes in the list have the same number of columns, in the same order.**

In [ ]:
str(df)

In [ ]:
names(df)

In [ ]:
str(df)

In [ ]:
meta.data <- do.call(rbind, df)
rownames(meta.data) = meta.data$Barcode
meta.data$Barcode <- NULL

In [ ]:
head(meta.data)

In [ ]:
dim(E12_5@meta.data)
dim(meta.data)

In [ ]:
meta.data$CellType2 <- NULL

In [ ]:
dim(E12_5@meta.data)
dim(meta.data)

In [ ]:
target <- rownames(E12_5@meta.data)
meta.data <- meta.data[match(target, rownames(meta.data)),]

In [ ]:
E12_5@meta.data = meta.data

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
rownames(E12_5@meta.data)

In [ ]:
Idents(E12_5) <- E12_5@meta.data$CellType

In [ ]:
E12_5 <- subset(E12_5, idents = c("Doublets"), invert = TRUE)

In [ ]:
options(repr.plot.width=10)
DimPlot(E12_5, group.by = "CellType", label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_a.png" ,width=10, height=8)

In [ ]:
options(repr.plot.width=10)
DimPlot(E12_5, label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_a.png" ,width=10, height=8)

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
Idents(E12_5) <- E12_5$CellType

In [ ]:
saveRDS(E12_5, file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E12_5_seuratObject_annotation.Rds")

In [ ]:
E12_5

In [ ]:
unique(Idents(E12_5))